In [11]:
import pandas as pd
import os

In [176]:
# Словарь датафреймов
dfs = {}

# Проход по всем файлам
for fl in os.listdir('src/'):
    if fl.endswith('.csv'):
        file_path = os.path.join('src/', fl)
        # Читаем в df
        df = pd.read_csv(file_path, sep = ';')
# Сохраняем датафрейм в словаре с именем без расширения
        dfs[os.path.splitext(fl)[0]] = df
# Хотя, лучше откажемся от данного варианта    

In [257]:
import pandas as pd

# Загрузка данных из CSV-файлов
contact_preference = pd.read_csv('src/contact_preference.csv', sep = ';')
customer_preferenced_contact_type = pd.read_csv('src/customer_preferenced_contact_type.csv', sep = ';')
customer = pd.read_csv('src/customer.csv', sep = ';')
customers_segments = pd.read_csv('src/customers_segments.csv', sep = ';')
location = pd.read_csv('src/location.csv', sep = ';')
product_feedback = pd.read_csv('src/product_feedback.csv', sep = ';')
product = pd.read_csv('src/product.csv', sep = ';')
purchase = pd.read_csv('src/purchase.csv', sep = ';')
segment = pd.read_csv('src/segment.csv', sep = ';')
service_feedback = pd.read_csv('src/service_feedback.csv', sep = ';')

# Объединение таблиц
# 1. Объединяем покупки с продуктами
purchase_product = purchase.merge(product, left_on='product_id', right_on='id', suffixes=('', '_product'))

# 2. Объединяем с отзывами по продуктам
purchase_product_feedback = purchase_product.merge(product_feedback, left_on='id', right_on='purchase_id', suffixes=('', '_feedback'))

# 3. Объединяем с клиентами для получения информации о пользователе
purchase_info = purchase_product_feedback.merge(customer, left_on='user_id', right_on='id', suffixes=('', '_customer'))

# 4. Получаем информацию о сегментах
purchase_info_segment = purchase_info.merge(customers_segments, left_on='user_id', right_on='customer_id', suffixes=('', '_segment'))

# 5. Объединяем с сегментами
purchase_info_segment = purchase_info_segment.merge(segment, left_on='segment_id', right_on='id', suffixes=('', '_segment_info'))

# Теперь мы можем агрегировать нужные метрики для витрины пользователей
user_summary = purchase_info_segment.groupby('user_id').agg(
    total_purchases=('id', 'count'),
    total_spent=('price', 'sum'),
    average_product_rating=('rating', 'mean'),
#     Вот тут ошибка, пока не придумал, как ее починить
#     highest_rated_product=('product_name', lambda x: x[product_feedback['rating'].idxmax()]), 
    most_popular_category=('category_name', lambda x: x.mode()[0]),
    total_reviews=('review', 'count')
).reset_index()

user_summary

,user_id,total_purchases,total_spent,average_product_rating,most_popular_category,total_reviews
0,1,6,14.9914.9949.9959.997.9910.99,8.000000,Games,6
1,2,8,14.9914.999.4912.4939.9959.9912.9914.99,8.125000,Books,8
2,3,4,19.9915.9912.9910.99,8.750000,Books,4
3,4,8,7.9919.9914.9915.9926.999.4914.9914.99,7.625000,Books,8
4,5,3,19.9914.998.49,9.000000,Books,3
5,6,4,49.9959.9914.9959.99,6.750000,Games,4
6,7,4,14.9910.997.9912.99,7.500000,Books,4
7,8,9,29.9929.9912.9959.9914.9929.9913.4911.9910.99,7.000000,Books,9
8,9,7,29.9912.9914.9919.9915.9914.9914.99,7.857143,Movies,7
9,10,3,12.9929.9912.99,8.000000,Books,3
